## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Glenwood Springs,US,39.5505,-107.3248,50.81,73,100,11.50,overcast clouds
1,1,Gladstone,AU,-23.8500,151.2500,66.13,72,3,4.61,clear sky
2,2,Ushuaia,AR,-54.8000,-68.3000,45.57,95,100,9.10,overcast clouds
3,3,Vaini,TO,-21.2000,-175.2000,77.16,94,40,4.61,scattered clouds
4,4,Saldanha,ZA,-33.0117,17.9442,73.45,78,2,12.26,clear sky
5,5,Khatanga,RU,71.9667,102.5000,9.55,95,99,11.54,overcast clouds
6,6,Sola,VU,-13.8833,167.5500,81.18,73,87,4.03,overcast clouds
7,7,Lorengau,PG,-2.0226,147.2712,81.09,81,62,4.94,broken clouds
8,8,Airai,TL,-8.9266,125.4092,60.48,91,100,1.90,overcast clouds
9,9,Barrow,US,71.2906,-156.7887,-5.78,84,0,18.41,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = int(input('What is the minimum temperature you want in your trip?'))
max_temp = int(input('What is the maximum temperature you want in your trip?'))

What is the minimum temperature you want in your trip?75
What is the maximum temperature you want in your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                 (city_data_df['Max Temp'] >= min_temp)]
preferred_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
3,3,Vaini,TO,-21.2000,-175.2000,77.16,94,40,4.61,scattered clouds
6,6,Sola,VU,-13.8833,167.5500,81.18,73,87,4.03,overcast clouds
7,7,Lorengau,PG,-2.0226,147.2712,81.09,81,62,4.94,broken clouds
11,11,Puerto Ayora,EC,-0.7393,-90.3518,78.76,84,93,8.97,light rain
16,16,Hithadhoo,MV,-0.6000,73.0833,83.48,75,65,7.78,broken clouds
17,17,Rikitea,PF,-23.1203,-134.9692,78.12,66,2,13.11,clear sky
21,21,George Town,MY,5.4112,100.3354,82.33,88,40,3.44,scattered clouds
30,30,Hobyo,SO,5.3505,48.5268,83.93,79,16,17.69,few clouds
32,32,Avarua,CK,-21.2078,-159.7750,75.25,94,40,3.44,scattered clouds
36,36,Wattegama,LK,6.7989,81.4808,75.06,93,24,3.42,few clouds


In [5]:
preferred_cities.shape

(231, 10)

In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities.isnull().sum()

City_ID        0
City           0
Country        4
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities = preferred_cities.dropna() #there weren't any null values so nothing changed
preferred_cities.count()

City_ID        227
City           227
Country        227
Lat            227
Lng            227
Max Temp       227
Humidity       227
Cloudiness     227
Wind Speed     227
Description    227
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,
6,Sola,VU,81.18,overcast clouds,-13.8833,167.5500,
7,Lorengau,PG,81.09,broken clouds,-2.0226,147.2712,
11,Puerto Ayora,EC,78.76,light rain,-0.7393,-90.3518,
16,Hithadhoo,MV,83.48,broken clouds,-0.6000,73.0833,
17,Rikitea,PF,78.12,clear sky,-23.1203,-134.9692,
21,George Town,MY,82.33,scattered clouds,5.4112,100.3354,
30,Hobyo,SO,83.93,few clouds,5.3505,48.5268,
32,Avarua,CK,75.25,scattered clouds,-21.2078,-159.7750,
36,Wattegama,LK,75.06,few clouds,6.7989,81.4808,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('No hotel found... skipping!')
        
        

No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!


In [10]:
hotel_df.count()

City           227
Country        227
Max Temp       227
Description    227
Lat            227
Lng            227
Hotel Name     227
dtype: int64

In [11]:
# 7. Drop the rows where there is no Hotel Name. We should expect 15 rows to drop
hotel_df[hotel_df['Hotel Name'] == ''] #This code tells us there are 15 rows without hotel name
hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
hotel_df.count() # Now we have 15 less rows by dropping the ones that didn't find any hotel

City           208
Country        208
Max Temp       208
Description    208
Lat            208
Lng            208
Hotel Name     208
dtype: int64

In [12]:
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
6,Sola,VU,81.18,overcast clouds,-13.8833,167.5500,Leumerus Bungalows
7,Lorengau,PG,81.09,broken clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
11,Puerto Ayora,EC,78.76,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel
16,Hithadhoo,MV,83.48,broken clouds,-0.6000,73.0833,Scoop Guest House
17,Rikitea,PF,78.12,clear sky,-23.1203,-134.9692,People ThankYou
21,George Town,MY,82.33,scattered clouds,5.4112,100.3354,Cititel Penang
30,Hobyo,SO,83.93,few clouds,5.3505,48.5268,Iftin hotel hobyo
32,Avarua,CK,75.25,scattered clouds,-21.2078,-159.7750,Paradise Inn
39,Mahebourg,MU,78.26,light intensity shower rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa


In [13]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temperature</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
print(fig)

Figure(layout=FigureLayout(height='420px'))
